In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/IITP/sohyun/creditcard_prediction/data

Mounted at /content/drive
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [3]:
import torch
import os
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image
import h5py
import numpy as np
import collections
import numbers
import math
import pandas as pd

class KDD99Loader(object):
    def __init__(self, data_path, mode="train"):
        self.mode=mode
        data = np.load(data_path)

        labels = data["kdd"][:,-1]
        features = data["kdd"][:,:-1]
        N, D = features.shape
        
        normal_data = features[labels==1]
        normal_labels = labels[labels==1]

        N_normal = normal_data.shape[0]

        attack_data = features[labels==0]
        attack_labels = labels[labels==0]

        N_attack = attack_data.shape[0]

        randIdx = np.arange(N_attack)
        np.random.shuffle(randIdx)
        N_train = N_attack // 2

        self.train = attack_data[randIdx[:N_train]]
        self.train_labels = attack_labels[randIdx[:N_train]]

        self.test = attack_data[randIdx[N_train:]]
        self.test_labels = attack_labels[randIdx[N_train:]]

        self.test = np.concatenate((self.test, normal_data),axis=0)
        self.test_labels = np.concatenate((self.test_labels, normal_labels),axis=0)


    def __len__(self):
        """
        Number of images in the object dataset.
        """
        if self.mode == "train":
            return self.train.shape[0]
        else:
            return self.test.shape[0]


    def __getitem__(self, index):
        if self.mode == "train":
            return np.float32(self.train[index]), np.float32(self.train_labels[index])
        else:
           return np.float32(self.test[index]), np.float32(self.test_labels[index])
        

def get_loader(data_path, batch_size, mode='train'):
    """Build and return data loader."""

    dataset = KDD99Loader(data_path, mode)

    shuffle = False
    if mode == 'train':
        shuffle = True

    data_loader = DataLoader(dataset=dataset,
                             batch_size=batch_size,
                             shuffle=shuffle)
    return data_loader

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torch.autograd import Variable
import itertools


class Cholesky(torch.autograd.Function):
    def forward(ctx, a):
        l = torch.cholesky(a, False)
        ctx.save_for_backward(l)
        return l
    def backward(ctx, grad_output):
        l, = ctx.saved_variables
        linv = l.inverse()
        inner = torch.tril(torch.mm(l.t(), grad_output)) * torch.tril(
            1.0 - Variable(l.data.new(l.size(1)).fill_(0.5).diag()))
        s = torch.mm(linv.t(), torch.mm(inner, linv))
        return s
    
class DaGMM(nn.Module):
    """Residual Block."""
    def __init__(self, n_gmm = 2, latent_dim=3):
        super(DaGMM, self).__init__()

        layers = []
        layers += [nn.Linear(118,60)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(60,30)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(30,10)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(10,1)]

        self.encoder = nn.Sequential(*layers)


        layers = []
        layers += [nn.Linear(1,10)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(10,30)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(30,60)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(60,118)]

        self.decoder = nn.Sequential(*layers)

        layers = []
        layers += [nn.Linear(latent_dim,10)]
        layers += [nn.Tanh()]        
        layers += [nn.Dropout(p=0.5)]        
        layers += [nn.Linear(10,n_gmm)]
        layers += [nn.Softmax(dim=1)]


        self.estimation = nn.Sequential(*layers)

        self.register_buffer("phi", torch.zeros(n_gmm))
        self.register_buffer("mu", torch.zeros(n_gmm,latent_dim))
        self.register_buffer("cov", torch.zeros(n_gmm,latent_dim,latent_dim))

    def relative_euclidean_distance(self, a, b):
        return (a-b).norm(2, dim=1) / a.norm(2, dim=1)

    def forward(self, x):

        enc = self.encoder(x)

        dec = self.decoder(enc)

        rec_cosine = F.cosine_similarity(x, dec, dim=1)
        rec_euclidean = self.relative_euclidean_distance(x, dec)

        z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)

        gamma = self.estimation(z)

        return enc, dec, z, gamma

    def compute_gmm_params(self, z, gamma):
        N = gamma.size(0)
        # K
        sum_gamma = torch.sum(gamma, dim=0)

        # K
        phi = (sum_gamma / N)

        self.phi = phi.data

 
        # K x D
        mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1)
        self.mu = mu.data
        # z = N x D
        # mu = K x D
        # gamma N x K

        # z_mu = N x K x D
        z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

        # z_mu_outer = N x K x D x D
        z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2)

        # K x D x D
        cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim = 0) / sum_gamma.unsqueeze(-1).unsqueeze(-1)
        self.cov = cov.data

        return phi, mu, cov
        
    def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
        if phi is None:
            phi = to_var(self.phi)
        if mu is None:
            mu = to_var(self.mu)
        if cov is None:
            cov = to_var(self.cov)

        k, D, _ = cov.size()

        z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

        cov_inverse = []
        det_cov = []
        cov_diag = 0
        eps = 1e-12
        for i in range(k):
            # K x D x D
            cov_k = cov[i] + to_var(torch.eye(D)*eps)
            cov_inverse.append(torch.inverse(cov_k).unsqueeze(0))

            #det_cov.append(np.linalg.det(cov_k.data.cpu().numpy()* (2*np.pi)))
            det_cov.append((Cholesky.apply(cov_k.cpu() * (2*np.pi)).diag().prod()).unsqueeze(0))
            cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

        # K x D x D
        cov_inverse = torch.cat(cov_inverse, dim=0)
        # K
        det_cov = torch.cat(det_cov).cuda()
        #det_cov = to_var(torch.from_numpy(np.float32(np.array(det_cov))))

        # N x K
        exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1) * cov_inverse.unsqueeze(0), dim=-2) * z_mu, dim=-1)
        # for stability (logsumexp)
        max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0]

        exp_term = torch.exp(exp_term_tmp - max_val)

        # sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (det_cov).unsqueeze(0), dim = 1) + eps)
        sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt(det_cov)).unsqueeze(0), dim = 1) + eps)
        # sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt((2*np.pi)**D * det_cov)).unsqueeze(0), dim = 1) + eps)


        if size_average:
            sample_energy = torch.mean(sample_energy)

        return sample_energy, cov_diag


    def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):

        recon_error = torch.mean((x - x_hat) ** 2)

        phi, mu, cov = self.compute_gmm_params(z, gamma)

        sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)

        loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag

        return loss, sample_energy, recon_error, cov_diag

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import time
import datetime
from torch.autograd import grad
from torch.autograd import Variable
import matplotlib.pyplot as plt
import IPython
from tqdm import tqdm

class Solver(object):
    DEFAULTS = {}   
    def __init__(self, data_loader, config):
        # Data loader
        self.__dict__.update(Solver.DEFAULTS, **config)
        self.data_loader = data_loader

        # Build tensorboard if use
        self.build_model()
        # if self.use_tensorboard:
        #     self.build_tensorboard()

        # Start with trained model
        if self.pretrained_model:
            self.load_pretrained_model()

    def build_model(self):
        # Define model
        self.dagmm = DaGMM(self.gmm_k)

        # Optimizers
        self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr)

        # Print networks
        self.print_network(self.dagmm, 'DaGMM')

        if torch.cuda.is_available():
            self.dagmm.cuda()

    def print_network(self, model, name):
        num_params = 0
        for p in model.parameters():
            num_params += p.numel()
        print(name)
        print(model)
        print("The number of parameters: {}".format(num_params))

    def load_pretrained_model(self):
        self.dagmm.load_state_dict(torch.load(os.path.join(
            self.model_save_path, '{}_dagmm.pth'.format(self.pretrained_model))))

        print("phi", self.dagmm.phi,"mu",self.dagmm.mu, "cov",self.dagmm.cov)

        print('loaded trained models (step: {})..!'.format(self.pretrained_model))

    def reset_grad(self):
        self.dagmm.zero_grad()

    def to_var(self, x, volatile=False):
        if torch.cuda.is_available():
            x = x.cuda()
        return Variable(x, volatile=volatile)

    def train(self):
        iters_per_epoch = len(self.data_loader)

        # Start with trained model if exists
        if self.pretrained_model:
            start = int(self.pretrained_model.split('_')[0])
        else:
            start = 0

        # Start training
        iter_ctr = 0
        start_time = time.time()



        self.ap_global_train = np.array([0,0,0])
        for e in range(start, self.num_epochs):
            for i, (input_data, labels) in enumerate(tqdm(self.data_loader)):
                iter_ctr += 1
                start = time.time()

                input_data = self.to_var(input_data)

                total_loss,sample_energy, recon_error, cov_diag = self.dagmm_step(input_data)
                # Logging
                loss = {}
                loss['total_loss'] = total_loss.data.item()
                loss['sample_energy'] = sample_energy.item()
                loss['recon_error'] = recon_error.item()
                loss['cov_diag'] = cov_diag.item()

                print("phi", self.dagmm.phi,"mu",self.dagmm.mu, "cov",self.dagmm.cov)
                # Save model checkpoints
                if (i+1) % self.model_save_step == 0:
                    torch.save(self.dagmm.state_dict(),
                        os.path.join(self.model_save_path, '{}_{}_dagmm.pth'.format(e+1, i+1)))

    def dagmm_step(self, input_data):
        self.dagmm.train()
        enc, dec, z, gamma = self.dagmm(input_data)

        total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma, self.lambda_energy, self.lambda_cov_diag)

        self.reset_grad()
        total_loss.backward()

        torch.nn.utils.clip_grad_norm_(self.dagmm.parameters(), 5)
        self.optimizer.step()

        return total_loss,sample_energy, recon_error, cov_diag

    def test(self):
        print("======================TEST MODE======================")
        self.dagmm.eval()
        self.data_loader.dataset.mode="train"

        N = 0
        mu_sum = 0
        cov_sum = 0
        gamma_sum = 0

        for it, (input_data, labels) in enumerate(self.data_loader):
            input_data = self.to_var(input_data)
            enc, dec, z, gamma = self.dagmm(input_data)
            phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)
            
            batch_gamma_sum = torch.sum(gamma, dim=0)
            
            gamma_sum += batch_gamma_sum
            mu_sum += mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
            cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
            
            N += input_data.size(0)
            
        train_phi = gamma_sum / N
        train_mu = mu_sum / gamma_sum.unsqueeze(-1)
        train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)

        # print("N:",N)
        # print("phi :\n",train_phi)
        # print("mu :\n",train_mu)
        # print("cov :\n",train_cov)

        train_energy = []
        train_labels = []
        train_z = []
        for it, (input_data, labels) in enumerate(self.data_loader):
            input_data = self.to_var(input_data)
            enc, dec, z, gamma = self.dagmm(input_data)
            sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
            
            train_energy.append(sample_energy.data.cpu().numpy())
            train_z.append(z.data.cpu().numpy())
            train_labels.append(labels.numpy())


        train_energy = np.concatenate(train_energy,axis=0)
        train_z = np.concatenate(train_z,axis=0)
        train_labels = np.concatenate(train_labels,axis=0)


        self.data_loader.dataset.mode="test"
        test_energy = []
        test_labels = []
        test_z = []
        for it, (input_data, labels) in enumerate(self.data_loader):
            input_data = self.to_var(input_data)
            enc, dec, z, gamma = self.dagmm(input_data)
            sample_energy, cov_diag = self.dagmm.compute_energy(z, size_average=False)
            test_energy.append(sample_energy.data.cpu().numpy())
            test_z.append(z.data.cpu().numpy())
            test_labels.append(labels.numpy())


        test_energy = np.concatenate(test_energy,axis=0)
        test_z = np.concatenate(test_z,axis=0)
        test_labels = np.concatenate(test_labels,axis=0)

        combined_energy = np.concatenate([train_energy, test_energy], axis=0)
        combined_labels = np.concatenate([train_labels, test_labels], axis=0)

        thresh = np.percentile(combined_energy, 100 - 20)
        print("Threshold :", thresh)

        pred = (test_energy > thresh).astype(int)
        gt = test_labels.astype(int)

        from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score

        accuracy = accuracy_score(gt,pred)
        precision, recall, f_score, support = prf(gt, pred, average='binary')

        print("Accuracy : {:0.4f}, Precision : {:0.4f}, Recall : {:0.4f}, F-score : {:0.4f}".format(accuracy, precision, recall, f_score))
        
        return accuracy, precision, recall, f_score

In [6]:
import os
import argparse
from torch.backends import cudnn
import easydict

def str2bool(v):
    return v.lower() in ('true')

def main(config):
    # For fast training
    cudnn.benchmark = True

    # Create directories if not exist
    mkdir(config.log_path)
    mkdir(config.model_save_path)

    data_loader = get_loader(config.data_path, batch_size=config.batch_size, mode=config.mode)
    
    # Solver
    solver = Solver(data_loader, vars(config))

    if config.mode == 'train':
        solver.train()
    elif config.mode == 'test':
        solver.test()

    return solver
    
if __name__ == '__main__':
    args = easydict.EasyDict({
        "num_epochs" : 1,
        "batch_size" : 1024,
        "gmm_k" : 4,
        "lambda_energy" : 0.1,
        "lambda_cov_diag" : 0.005,
        # "pretrained_model" : '',
        "pretrained_model" : None,
        "mode" : 'train',
        # "mode" : "test",
        "data_path" : "./kdd_cup.npz",   
        "use_tensorboard" : False,
        "log_path" : './logs',
        "model_save_path" : './models',
        "log_step" : 194//4,
        "sample_step" : 194,
        "model_save_step" : 194,
        "lr" : 1e-4,
        "wd" : None
    })
    config = args
 
    print('------------ Options -------------')
    for k, v in sorted(args.items()):
        print('%s: %s' % (str(k), str(v)))
    print('-------------- End ----------------')

    solver = main(config)
    solver.test()

------------ Options -------------
batch_size: 1024
data_path: ./kdd_cup.npz
gmm_k: 4
lambda_cov_diag: 0.005
lambda_energy: 0.1
log_path: ./logs
log_step: 48
lr: 0.0001
mode: train
model_save_path: ./models
model_save_step: 194
num_epochs: 1
pretrained_model: None
sample_step: 194
use_tensorboard: False
wd: None
-------------- End ----------------
DaGMM
DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=118, out_features=60, bias=True)
    (1): Tanh()
    (2): Linear(in_features=60, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=10, bias=True)
    (5): Tanh()
    (6): Linear(in_features=10, out_features=1, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=1, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=60, bias=True)
    (5): Tanh()
    (6): Linear(in_features=60, out_features=118, bias=True)
  )
  (

  0%|          | 0/194 [00:00<?, ?it/s]<ipython-input-4-ffe31a3e940e>:12: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1615.)
  l = torch.cholesky(a, False)
<ipython-input-4-ffe31a3e940e>:16: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  l, = ctx.saved_variables
  3%|▎         | 6/194 [00:04<01:48,  1.73it/s]

phi tensor([0.3440, 0.1887, 0.2436, 0.2237], device='cuda:0') mu tensor([[-0.0023,  1.0944,  0.0717],
        [-0.0029,  1.0940,  0.0722],
        [-0.0027,  1.0937,  0.0728],
        [-0.0026,  1.0941,  0.0720]], device='cuda:0') cov tensor([[[ 0.0015,  0.0011, -0.0015],
         [ 0.0011,  0.0011, -0.0017],
         [-0.0015, -0.0017,  0.0026]],

        [[ 0.0015,  0.0010, -0.0015],
         [ 0.0010,  0.0011, -0.0016],
         [-0.0015, -0.0016,  0.0026]],

        [[ 0.0015,  0.0010, -0.0015],
         [ 0.0010,  0.0011, -0.0016],
         [-0.0015, -0.0016,  0.0025]],

        [[ 0.0015,  0.0011, -0.0015],
         [ 0.0011,  0.0011, -0.0016],
         [-0.0015, -0.0016,  0.0026]]], device='cuda:0')
phi tensor([0.3433, 0.1890, 0.2460, 0.2217], device='cuda:0') mu tensor([[-0.0064,  1.0918,  0.0770],
        [-0.0064,  1.0924,  0.0756],
        [-0.0061,  1.0925,  0.0756],
        [-0.0060,  1.0921,  0.0763]], device='cuda:0') cov tensor([[[ 0.0016,  0.0011, -0.0016],
         [ 

  8%|▊         | 15/194 [00:04<00:31,  5.72it/s]

phi tensor([0.3391, 0.1924, 0.2466, 0.2219], device='cuda:0') mu tensor([[-0.0198,  1.0891,  0.0858],
        [-0.0205,  1.0890,  0.0858],
        [-0.0197,  1.0890,  0.0858],
        [-0.0199,  1.0897,  0.0846]], device='cuda:0') cov tensor([[[ 0.0026,  0.0020, -0.0032],
         [ 0.0020,  0.0019, -0.0031],
         [-0.0032, -0.0031,  0.0051]],

        [[ 0.0026,  0.0020, -0.0031],
         [ 0.0020,  0.0019, -0.0031],
         [-0.0031, -0.0031,  0.0052]],

        [[ 0.0026,  0.0020, -0.0032],
         [ 0.0020,  0.0019, -0.0031],
         [-0.0032, -0.0031,  0.0051]],

        [[ 0.0026,  0.0020, -0.0032],
         [ 0.0020,  0.0019, -0.0032],
         [-0.0032, -0.0032,  0.0054]]], device='cuda:0')
phi tensor([0.3399, 0.1911, 0.2468, 0.2221], device='cuda:0') mu tensor([[-0.0212,  1.0870,  0.0902],
        [-0.0213,  1.0871,  0.0900],
        [-0.0209,  1.0870,  0.0902],
        [-0.0216,  1.0868,  0.0905]], device='cuda:0') cov tensor([[[ 0.0029,  0.0021, -0.0034],
         [ 

...

 94%|█████████▍| 183/194 [00:10<00:00, 35.26it/s]

phi tensor([0.3756, 0.1847, 0.2514, 0.1884], device='cuda:0') mu tensor([[-0.3037,  0.8777,  0.5264],
        [-0.2748,  0.8993,  0.4989],
        [-0.2756,  0.8988,  0.4997],
        [-0.2459,  0.9209,  0.4715]], device='cuda:0') cov tensor([[[ 0.2021,  0.1513, -0.1918],
         [ 0.1513,  0.1133, -0.1436],
         [-0.1918, -0.1436,  0.1820]],

        [[ 0.2156,  0.1614, -0.2046],
         [ 0.1614,  0.1209, -0.1532],
         [-0.2046, -0.1532,  0.1942]],

        [[ 0.2157,  0.1615, -0.2047],
         [ 0.1615,  0.1210, -0.1533],
         [-0.2047, -0.1533,  0.1943]],

        [[ 0.2277,  0.1705, -0.2161],
         [ 0.1705,  0.1277, -0.1618],
         [-0.2161, -0.1618,  0.2051]]], device='cuda:0')
phi tensor([0.3801, 0.1839, 0.2487, 0.1873], device='cuda:0') mu tensor([[-0.2871,  0.8896,  0.5117],
        [-0.2518,  0.9159,  0.4784],
        [-0.2606,  0.9094,  0.4866],
        [-0.2400,  0.9250,  0.4670]], device='cuda:0') cov tensor([[[ 0.2113,  0.1582, -0.1999],
         [ 

100%|██████████| 194/194 [00:10<00:00, 18.74it/s]

phi tensor([0.3801, 0.1834, 0.2494, 0.1872], device='cuda:0') mu tensor([[-0.3102,  0.8678,  0.5386],
        [-0.2815,  0.8891,  0.5118],
        [-0.2889,  0.8836,  0.5188],
        [-0.2633,  0.9028,  0.4949]], device='cuda:0') cov tensor([[[ 0.2128,  0.1594, -0.1994],
         [ 0.1594,  0.1194, -0.1494],
         [-0.1994, -0.1494,  0.1869]],

        [[ 0.2269,  0.1698, -0.2126],
         [ 0.1698,  0.1272, -0.1592],
         [-0.2126, -0.1592,  0.1992]],

        [[ 0.2235,  0.1673, -0.2094],
         [ 0.1673,  0.1253, -0.1568],
         [-0.2094, -0.1568,  0.1962]],

        [[ 0.2354,  0.1762, -0.2205],
         [ 0.1762,  0.1320, -0.1651],
         [-0.2205, -0.1651,  0.2066]]], device='cuda:0')
phi tensor([0.3805, 0.1802, 0.2533, 0.1860], device='cuda:0') mu tensor([[-0.3162,  0.8621,  0.5451],
        [-0.2959,  0.8773,  0.5262],
        [-0.2844,  0.8859,  0.5154],
        [-0.2637,  0.9011,  0.4962]], device='cuda:0') cov tensor([[[ 0.2089,  0.1560, -0.1952],
         [ 

======================TEST MODE======================
N: 198371
phi :
 tensor([0.3841, 0.1834, 0.2488, 0.1836], device='cuda:0',
       grad_fn=<DivBackward0>)
mu :
 tensor([[-0.3056,  0.8690,  0.5366],
        [-0.2746,  0.8921,  0.5080],
        [-0.2710,  0.8948,  0.5047],
        [-0.2433,  0.9156,  0.4791]], device='cuda:0', grad_fn=<DivBackward0>)
cov :
 tensor([[[ 0.2276,  0.1704, -0.2101],
         [ 0.1704,  0.1276, -0.1573],
         [-0.2101, -0.1573,  0.1940]],

        [[ 0.2427,  0.1817, -0.2240],
         [ 0.1817,  0.1361, -0.1678],
         [-0.2240, -0.1678,  0.2068]],

        [[ 0.2443,  0.1829, -0.2255],
         [ 0.1829,  0.1370, -0.1688],
         [-0.2255, -0.1688,  0.2082]],

        [[ 0.2561,  0.1917, -0.2364],
         [ 0.1917,  0.1436, -0.1770],
         [-0.2364, -0.1770,  0.2183]]], device='cuda:0',
       grad_fn=<DivBackward0>)
Threshold : 2.341646909713745
Accuracy : 0.9556, Precision : 0.9447, Recall : 0.9188, F-score : 0.9316
